In [1]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from google.colab import drive

import numpy as np

import os
import time
import random

In [2]:
# drive.mount('/content/drive/')
# !unzip -q drive/MyDrive/data/data2/samples.zip -d data
# !cp drive/MyDrive/data/data2/labels.npy data/

Mounted at /content/drive/


In [3]:
# https://github.com/ndrplz/ConvLSTM_pytorch/blob/master/convlstm.py


class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = T.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = T.split(combined_conv, self.hidden_dim, dim=1)
        i = T.sigmoid(cc_i)
        f = T.sigmoid(cc_f)
        o = T.sigmoid(cc_o)
        g = T.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * T.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (T.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                T.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):

    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = T.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """
        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful
        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = T.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [4]:

class Data(Dataset):

  def __init__(self, samples, labels):
    self.samples = samples
    self.labels = labels
    self.len = samples.size(0)
  
  def __getitem__(self, index):
    return self.samples[index, :, :], self.labels[index, :]

  def __len__(self):
    return self.len


class LazyData(Dataset):

  def __init__(self, samples_path, labels_path):
    self.samples_path = samples_path
    self.labels_path = labels_path
    self.samples_list = sorted(
      os.listdir(samples_path),
      key=lambda x: int(os.path.splitext(x)[0])
    )
    labels = np.load(labels_path)
    self.labels = T.tensor(labels)
    self.len = len(self.samples_list)
  
  def __getitem__(self, index):
    sample_path = os.path.join(self.samples_path, self.samples_list[index])
    sample = np.load(sample_path)
    sample = T.from_numpy(sample)
    label = self.labels[index, :]
    return sample, label

  def __len__(self):
    return self.len


class SmallData(Dataset):

  def __init__(self, samples_path, labels_path, n_samples=1):
    self.samples_path = samples_path
    self.labels_path = labels_path
    all_samples_names = sorted(
      os.listdir(samples_path),
      key=lambda x: int(os.path.splitext(x)[0])
    )
    all_labels = np.load(labels_path)

    self.len = min(n_samples, len(all_samples_names))
    keep_indices = random.choices(
      range(self.len), 
      k=n_samples
    )

    samples_numpy = np.empty((n_samples, 15, 3, 224, 224), dtype=np.single)
    labels_numpy = np.empty((n_samples, 1), dtype=np.single)
    for i, k in enumerate(keep_indices):
      cur_sample_path = os.path.join(samples_path, all_samples_names[k])
      cur_sample = np.load(cur_sample_path)
      samples_numpy[i] = cur_sample
      labels_numpy[i] = all_labels[k]

    self.samples = T.as_tensor(samples_numpy)
    self.labels = T.as_tensor(labels_numpy)
  
  def __getitem__(self, index):
    sample = self.samples[index]
    label = self.labels[index]
    return sample, label

  def __len__(self):
    return self.len



In [5]:
def pram():
  free, total = T.cuda.mem_get_info()
  print(f'{free/1024**2:.2f} free of {total/1024**2:.2f}.')


class ConvLSTMBlock(nn.Module):

  def __init__(self, input_dim, hidden_dim, num_layers,
    conv_kernel_size=(3, 3), pool_kernel_size=(1, 2, 2)):
    
    super(ConvLSTMBlock, self).__init__()
    self.convlstm = ConvLSTM(
      input_dim=input_dim,
      hidden_dim=hidden_dim,
      kernel_size=conv_kernel_size,
      num_layers=num_layers,
      bias=False,
      batch_first=True
    )
    self.avgpool = nn.AvgPool3d(kernel_size=pool_kernel_size)
  
  def forward(self, x):
    x = self.convlstm(x)[0][0]
    x = T.permute(x, (0, 2, 1, 3, 4))
    x = self.avgpool(x)
    x = T.permute(x, (0, 2, 1, 3, 4))
    return x


class Model(nn.Module):

  def __init__(self):
    super(Model, self).__init__()

    self.clstm1 = ConvLSTMBlock(
      input_dim=3,
      hidden_dim=24,
      num_layers=1,
      conv_kernel_size=(5, 5),
      pool_kernel_size=(1, 4, 4)
    )

    self.clstm2 = ConvLSTMBlock(
      input_dim=24,
      hidden_dim=48,
      num_layers=1,
      conv_kernel_size=(5, 5),
      pool_kernel_size=(1, 3, 3)
    )

    self.clstm3 = ConvLSTMBlock(
      input_dim=48,
      hidden_dim=128,
      num_layers=1,
      conv_kernel_size=(3, 3),
      pool_kernel_size=(1, 3, 3)
    )

    self.clstm4 = ConvLSTMBlock(
      input_dim=128,
      hidden_dim=256,
      num_layers=1,
      conv_kernel_size=(3, 3),
      pool_kernel_size=(1, 2, 2)
    )

    self.conv = nn.Conv2d(
      in_channels=256, 
      out_channels=1024,
      kernel_size=(3, 3)
    )
    
    self.fc1 = nn.Linear(in_features=1024, out_features=1024)
    self.fc2 = nn.Linear(in_features=1024, out_features=512)
    self.fc3 = nn.Linear(in_features=512, out_features=256)
    self.fc4 = nn.Linear(in_features=256, out_features=1)
  
  def forward(self, x):

    x = self.clstm1(x)
    x = F.relu(x)
    x = self.clstm2(x)
    x = F.relu(x)
    x = self.clstm3(x)
    x = F.relu(x)
    x = self.clstm4(x)

    x = x[:, -1, :, :, :]
    x = self.conv(x)

    x = T.flatten(x, start_dim=1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    x = F.relu(x)
    x = self.fc3(x)
    x = F.relu(x)
    x = self.fc4(x)

    return x
  

In [6]:

def train_step(dataloader, device, model, criterion, optimizer):
  total_loss = 0
  len_counter = 0
  for x, y in dataloader:
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()
    total_loss += x.size(0) * loss.item()
    len_counter += x.size(0)
  total_loss /= len_counter
  return total_loss


def eval_step(dataloader, device, model, criterion):
  total_loss = 0
  len_counter = 0
  with T.no_grad():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      pred = model(x)
      loss = criterion(pred, y)
      total_loss += x.size(0) * loss.item() 
      len_counter += x.size(0)
  total_loss /= len_counter
  return total_loss


def train_loop(device, model, criterion, optimizer,
    train_loader, val_loader, test_loader,
    scheduler=None, init_epochs=100, add_epochs=False, verbose=False):

  model = model.to(device)
  if verbose:
    print(model)

  n_epochs = init_epochs
  total_epochs = init_epochs
  
  while n_epochs:

    for epoch in range(n_epochs):

      since = time.perf_counter()

      model.train()
      train_loss = train_step(train_loader, device, model, criterion, optimizer)

      model.eval()
      val_loss = eval_step(val_loader, device, model, criterion)

      if scheduler:
        scheduler.step(val_loss)

      if verbose:
        print(
          f'{time.perf_counter() - since:>5.1f}s :: ' +
          f'Epoch {total_epochs-n_epochs+epoch+1}/{total_epochs} complete.'
        )
        free_mem, total_mem = T.cuda.mem_get_info()
        print(
          ' ' * 10 +
          f'Device(free: {free_mem/(1024**2):.3f}, ' +
          f'total: {total_mem/(1024**2):.3f}).'
        )
        print(
          ' ' * 10 +
          f'Loss(train: {train_loss:.3f}, val: {val_loss:.3f}).'
        )

    test_loss = eval_step(test_loader, device, model, criterion)

    if verbose or add_epochs:
      print(f'Test loss: {test_loss}.')
    if add_epochs:
      n_epochs = input('Add epochs?')
      try:
        n_epochs = max(0, int(n_epochs))
        total_epochs += n_epochs
      except ValueError:
        n_epochs = 0
    else:
      n_epochs = 0


In [7]:

device0 = T.device('cuda' if T.cuda.is_available() else 'cpu')
print(f'Using device: {device0}.')

prop_val = 0.2
prop_test = 0.2

full_dataset = LazyData(
  os.path.join('data', 'data2'),
  os.path.join('data', 'labels.npy')
)
len_val = int(prop_val * len(full_dataset))
len_test = int(prop_test * len(full_dataset))
len_train = len(full_dataset) - len_val - len_test
train, val, test = random_split(
  full_dataset, [len_train, len_val, len_test],
  generator=T.Generator().manual_seed(4)
)


Using device: cuda.


In [ ]:

train_loader0 = DataLoader(train, batch_size=8, num_workers=4, shuffle=True)
val_loader0 = DataLoader(val, batch_size=8, num_workers=4)
test_loader0 = DataLoader(test, batch_size=8, num_workers=4)

model0 = Model()

criterion0 = nn.MSELoss()
optimizer0 = T.optim.Adam(model0.parameters(), lr=0.01)
scheduler0 = T.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer0, factor=0.3, verbose=True, patience=10
)

train_loop(
  device0, model0, criterion0, optimizer0,
  train_loader0, val_loader0, test_loader0,
  scheduler=scheduler0, add_epochs=True, verbose=True,
  init_epochs=201
)

Model(
  (clstm1): ConvLSTMBlock(
    (convlstm): ConvLSTM(
      (cell_list): ModuleList(
        (0): ConvLSTMCell(
          (conv): Conv2d(27, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
        )
      )
    )
    (avgpool): AvgPool3d(kernel_size=(1, 4, 4), stride=(1, 4, 4), padding=0)
  )
  (clstm2): ConvLSTMBlock(
    (convlstm): ConvLSTM(
      (cell_list): ModuleList(
        (0): ConvLSTMCell(
          (conv): Conv2d(72, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), bias=False)
        )
      )
    )
    (avgpool): AvgPool3d(kernel_size=(1, 3, 3), stride=(1, 3, 3), padding=0)
  )
  (clstm3): ConvLSTMBlock(
    (convlstm): ConvLSTM(
      (cell_list): ModuleList(
        (0): ConvLSTMCell(
          (conv): Conv2d(176, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
    )
    (avgpool): AvgPool3d(kernel_size=(1, 3, 3), stride=(1, 3, 3), padding=0)
  )
  (clstm4): ConvLSTMBlock(
    (convlstm): ConvLSTM(
 